# GroupKFold (5 splits) Validation Experiment

**HYPOTHESIS**: The CV-LB gap might be due to the validation scheme (Leave-One-Out vs GroupKFold).

**Observation**: The "mixall" kernel uses GroupKFold (5 splits) instead of Leave-One-Out.

**Why this might work**:
- Leave-One-Out CV is pessimistic (each fold has only 1 solvent in test)
- GroupKFold (5 splits) has ~5 solvents per test fold
- More test samples per fold → more stable estimates
- The CV-LB relationship might be different (lower intercept)

**Implementation**:
- Use the best model: GP(0.15) + MLP(0.55) + LGBM(0.3)
- Replace Leave-One-Out with GroupKFold (5 splits)
- Compare CV scores to see if the relationship changes

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import lightgbm as lgb
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
import tqdm
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

# NEW: GroupKFold (5 splits) instead of Leave-One-Out
def generate_groupkfold_splits(X, Y, n_splits=5):
    """GroupKFold with 5 splits instead of Leave-One-Out"""
    groups = X["SOLVENT NAME"]
    n_unique = len(groups.unique())
    actual_splits = min(n_splits, n_unique)
    gkf = GroupKFold(n_splits=actual_splits)
    for train_idx, test_idx in gkf.split(X, Y, groups):
        yield (X.iloc[train_idx], Y.iloc[train_idx]), (X.iloc[test_idx], Y.iloc[test_idx])

def generate_groupkfold_ramp_splits(X, Y, n_splits=5):
    """GroupKFold for mixture data - group by solvent pair"""
    # Create a unique identifier for each solvent pair
    groups = X["SOLVENT A NAME"] + "_" + X["SOLVENT B NAME"]
    n_unique = len(groups.unique())
    actual_splits = min(n_splits, n_unique)
    gkf = GroupKFold(n_splits=actual_splits)
    for train_idx, test_idx in gkf.split(X, Y, groups):
        yield (X.iloc[train_idx], Y.iloc[train_idx]), (X.iloc[test_idx], Y.iloc[test_idx])

# Keep original functions for comparison
def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = (X["SOLVENT A NAME"] != row["SOLVENT A NAME"]) | (X["SOLVENT B NAME"] != row["SOLVENT B NAME"])
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print("Data loading functions defined with GroupKFold (5 splits)")

Data loading functions defined with GroupKFold (5 splits)


In [3]:
# Load feature lookups
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
ACS_PCA_DF = pd.read_csv(f'{DATA_PATH}/acs_pca_descriptors_lookup.csv', index_col=0)

# Filter DRFP to high-variance columns
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'Spange: {SPANGE_DF.shape}, DRFP filtered: {DRFP_FILTERED.shape}, ACS PCA: {ACS_PCA_DF.shape}')

Spange: (26, 13), DRFP filtered: (24, 122), ACS PCA: (24, 5)


In [4]:
# Full Featurizer (for MLP and LGBM) - 145 features
class FullFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.drfp_df = DRFP_FILTERED
        self.acs_pca_df = ACS_PCA_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1] + self.drfp_df.shape[1] + self.acs_pca_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            A_acs = self.acs_pca_df.loc[X["SOLVENT A NAME"]].values
            B_acs = self.acs_pca_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
                X_drfp = B_drfp * (1 - (1-pct)) + A_drfp * (1-pct)
                X_acs = B_acs * (1 - (1-pct)) + A_acs * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
                X_drfp = A_drfp * (1 - pct) + B_drfp * pct
                X_acs = A_acs * (1 - pct) + B_acs * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
            X_acs = self.acs_pca_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange, X_drfp, X_acs])

    def featurize_torch(self, X, flip=False):
        return torch.tensor(self.featurize(X, flip), dtype=torch.double)

print(f'Full feature dimension: {FullFeaturizer().feats_dim}')

Full feature dimension: 145


In [5]:
# Simple Featurizer (for GP) - 18 features (Spange + Arrhenius kinetics)
class SimpleFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1]  # 18 features

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange])

print(f'Simple feature dimension (for GP): {SimpleFeaturizer().feats_dim}')

Simple feature dimension (for GP): 18


In [6]:
# Gaussian Process Wrapper
class GPWrapper:
    def __init__(self, data='single'):
        self.data_type = data
        self.featurizer = SimpleFeaturizer(mixed=(data=='full'))
        self.models = []
        self.scaler = None

    def train_model(self, X_train, y_train):
        X_std = self.featurizer.featurize(X_train, flip=False)
        y_vals = y_train.values
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize(X_train, flip=True)
            X_all = np.vstack([X_std, X_flip])
            y_all = np.vstack([y_vals, y_vals])
        else:
            X_all, y_all = X_std, y_vals
        
        self.scaler = StandardScaler()
        X_scaled = self.scaler.fit_transform(X_all)
        
        self.models = []
        kernel = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=2.5) + WhiteKernel(noise_level=0.1)
        
        for i in range(3):
            gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=3, random_state=42)
            gp.fit(X_scaled, y_all[:, i])
            self.models.append(gp)

    def predict(self, X_test):
        X_std = self.featurizer.featurize(X_test, flip=False)
        X_scaled = self.scaler.transform(X_std)
        
        preds = []
        for model in self.models:
            pred = model.predict(X_scaled)
            preds.append(pred)
        
        return torch.tensor(np.column_stack(preds), dtype=torch.double)

print('GPWrapper defined with Matern kernel')

GPWrapper defined with Matern kernel


In [7]:
# Weighted Huber Loss
class WeightedHuberLoss(nn.Module):
    def __init__(self, weights=[1.0, 1.0, 2.0]):
        super().__init__()
        self.weights = torch.tensor(weights, dtype=torch.double)
        self.huber = nn.HuberLoss(reduction='none')
    
    def forward(self, pred, target):
        huber_loss = self.huber(pred, target)
        weighted_loss = huber_loss * self.weights.to(pred.device)
        return weighted_loss.mean()

# MLP Model
class MLPModelInternal(nn.Module):
    def __init__(self, input_dim, hidden_dims=[32, 16], output_dim=3, dropout=0.05):
        super(MLPModelInternal, self).__init__()
        layers = [nn.BatchNorm1d(input_dim)]
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.extend([nn.Linear(prev_dim, h_dim), nn.BatchNorm1d(h_dim), nn.ReLU(), nn.Dropout(dropout)])
            prev_dim = h_dim
        layers.extend([nn.Linear(prev_dim, output_dim), nn.Sigmoid()])
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

print('MLP components defined')

MLP components defined


In [8]:
# MLP Ensemble with Weighted Loss
class WeightedMLPEnsemble:
    def __init__(self, hidden_dims=[32, 16], n_models=5, data='single', loss_weights=[1.0, 1.0, 2.0]):
        self.hidden_dims = hidden_dims
        self.n_models = n_models
        self.data_type = data
        self.loss_weights = loss_weights
        self.featurizer = FullFeaturizer(mixed=(data=='full'))
        self.models = []
        self.scalers = []

    def train_model(self, X_train, y_train, epochs=200, batch_size=32, lr=5e-4):
        X_std = self.featurizer.featurize_torch(X_train, flip=False)
        y_vals = torch.tensor(y_train.values)
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize_torch(X_train, flip=True)
            X_all = torch.cat([X_std, X_flip], dim=0)
            y_all = torch.cat([y_vals, y_vals], dim=0)
        else:
            X_all, y_all = X_std, y_vals
            
        input_dim = X_all.shape[1]
        self.models = []
        self.scalers = []
        
        for i in range(self.n_models):
            torch.manual_seed(42 + i)
            scaler = StandardScaler()
            X_scaled = torch.tensor(scaler.fit_transform(X_all.numpy()), dtype=torch.double)
            self.scalers.append(scaler)
            
            model = MLPModelInternal(input_dim, self.hidden_dims, dropout=0.05).double().to(device)
            optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=20)
            criterion = WeightedHuberLoss(self.loss_weights)
            
            dataset = TensorDataset(X_scaled, y_all)
            loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            
            model.train()
            for epoch in range(epochs):
                epoch_loss = 0
                for X_batch, y_batch in loader:
                    X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                    optimizer.zero_grad()
                    pred = model(X_batch)
                    loss = criterion(pred, y_batch)
                    loss.backward()
                    optimizer.step()
                    epoch_loss += loss.item()
                scheduler.step(epoch_loss)
            
            self.models.append(model)

    def predict(self, X_test):
        X_std = self.featurizer.featurize_torch(X_test, flip=False)
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize_torch(X_test, flip=True)
            preds = []
            for model, scaler in zip(self.models, self.scalers):
                model.eval()
                with torch.no_grad():
                    X_std_scaled = torch.tensor(scaler.transform(X_std.numpy()), dtype=torch.double).to(device)
                    X_flip_scaled = torch.tensor(scaler.transform(X_flip.numpy()), dtype=torch.double).to(device)
                    pred_std = model(X_std_scaled)
                    pred_flip = model(X_flip_scaled)
                    pred = (pred_std + pred_flip) / 2  # TTA
                    preds.append(pred.cpu())
            return torch.stack(preds).mean(dim=0)
        else:
            preds = []
            for model, scaler in zip(self.models, self.scalers):
                model.eval()
                with torch.no_grad():
                    X_scaled = torch.tensor(scaler.transform(X_std.numpy()), dtype=torch.double).to(device)
                    pred = model(X_scaled)
                    preds.append(pred.cpu())
            return torch.stack(preds).mean(dim=0)

print('WeightedMLPEnsemble defined')

WeightedMLPEnsemble defined


In [9]:
# LightGBM Wrapper
class LGBMWrapper:
    def __init__(self, data='single'):
        self.data_type = data
        self.featurizer = FullFeaturizer(mixed=(data=='full'))
        self.models = []
        self.scaler = None

    def train_model(self, X_train, y_train):
        X_std = self.featurizer.featurize(X_train, flip=False)
        y_vals = y_train.values
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize(X_train, flip=True)
            X_all = np.vstack([X_std, X_flip])
            y_all = np.vstack([y_vals, y_vals])
        else:
            X_all, y_all = X_std, y_vals
        
        self.scaler = StandardScaler()
        X_scaled = self.scaler.fit_transform(X_all)
        
        self.models = []
        params = {'objective': 'regression', 'metric': 'mse', 'boosting_type': 'gbdt',
                  'num_leaves': 31, 'learning_rate': 0.05, 'feature_fraction': 0.9,
                  'bagging_fraction': 0.8, 'bagging_freq': 5, 'verbose': -1, 'seed': 42}
        
        for i in range(3):
            train_data = lgb.Dataset(X_scaled, label=y_all[:, i])
            model = lgb.train(params, train_data, num_boost_round=500)
            self.models.append(model)

    def predict(self, X_test):
        X_std = self.featurizer.featurize(X_test, flip=False)
        X_scaled = self.scaler.transform(X_std)
        
        preds = []
        for model in self.models:
            pred = model.predict(X_scaled)
            preds.append(pred)
        
        return torch.tensor(np.column_stack(preds), dtype=torch.double)

print('LGBMWrapper defined')

LGBMWrapper defined


In [10]:
# GP + MLP + LGBM Ensemble (best weights from exp_035)
class GPMLPLGBMEnsemble:
    def __init__(self, data='single'):
        self.data_type = data
        self.gp = GPWrapper(data=data)
        self.mlp = WeightedMLPEnsemble(hidden_dims=[32, 16], n_models=5, data=data, loss_weights=[1.0, 1.0, 2.0])
        self.lgbm = LGBMWrapper(data=data)
        # Best weights from exp_035: GP 0.15, MLP 0.55, LGBM 0.3
        self.weights = {'gp': 0.15, 'mlp': 0.55, 'lgbm': 0.3}

    def train_model(self, X_train, y_train):
        self.gp.train_model(X_train, y_train)
        self.mlp.train_model(X_train, y_train)
        self.lgbm.train_model(X_train, y_train)

    def predict(self, X_test):
        gp_pred = self.gp.predict(X_test)
        mlp_pred = self.mlp.predict(X_test)
        lgbm_pred = self.lgbm.predict(X_test)
        
        combined = (self.weights['gp'] * gp_pred + 
                    self.weights['mlp'] * mlp_pred + 
                    self.weights['lgbm'] * lgbm_pred)
        return torch.clamp(combined, 0, 1)

print('GPMLPLGBMEnsemble defined: GP(0.15) + MLP(0.55) + LGBM(0.3)')

GPMLPLGBMEnsemble defined: GP(0.15) + MLP(0.55) + LGBM(0.3)


In [11]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

# CHANGE: Use GroupKFold (5 splits) instead of Leave-One-Out
split_generator = generate_groupkfold_splits(X, Y, n_splits=5)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = GPMLPLGBMEnsemble(data='single')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save fold predictions
submission_single_solvent = pd.DataFrame(all_predictions)
print(f"Single solvent predictions: {len(submission_single_solvent)} rows")

0it [00:00, ?it/s]

1it [00:55, 55.85s/it]

2it [01:43, 51.00s/it]

3it [02:26, 47.44s/it]

4it [03:10, 45.84s/it]

5it [03:54, 45.29s/it]

5it [03:54, 46.87s/it]

Single solvent predictions: 656 rows


In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

# CHANGE: Use GroupKFold (5 splits) instead of Leave-One-Ramp-Out
split_generator = generate_groupkfold_ramp_splits(X, Y, n_splits=5)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = GPMLPLGBMEnsemble(data='full')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save fold predictions
submission_full_data = pd.DataFrame(all_predictions)
print(f"Full data predictions: {len(submission_full_data)} rows")

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
# Calculate CV score (for verification only - NOT part of submission)
# NOTE: With GroupKFold, we need to match predictions to actuals differently

X_single, Y_single = load_data("single_solvent")
X_full, Y_full = load_data("full")

# For GroupKFold, we need to reconstruct the order of predictions
# The predictions are stored in fold order, so we need to match them

# Get actuals in the same order as GroupKFold splits
actuals_single = []
for fold_idx, ((train_X, train_Y), (test_X, test_Y)) in enumerate(generate_groupkfold_splits(X_single, Y_single, n_splits=5)):
    actuals_single.append(test_Y.values)
actuals_single = np.vstack(actuals_single)

actuals_full = []
for fold_idx, ((train_X, train_Y), (test_X, test_Y)) in enumerate(generate_groupkfold_ramp_splits(X_full, Y_full, n_splits=5)):
    actuals_full.append(test_Y.values)
actuals_full = np.vstack(actuals_full)

# Get predictions
preds_single = submission_single_solvent[['target_1', 'target_2', 'target_3']].values
preds_full = submission_full_data[['target_1', 'target_2', 'target_3']].values

# Calculate MSE
mse_single = np.mean((actuals_single - preds_single) ** 2)
mse_full = np.mean((actuals_full - preds_full) ** 2)

# Weighted average (same as competition)
n_single = len(actuals_single)
n_full = len(actuals_full)
mse_overall = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f"\n=== CV SCORE VERIFICATION (GroupKFold 5 splits) ===")
print(f"Single Solvent MSE: {mse_single:.6f} (n={n_single})")
print(f"Full Data MSE: {mse_full:.6f} (n={n_full})")
print(f"Overall MSE: {mse_overall:.6f}")

print(f"\n=== COMPARISON ===")
print(f"exp_035 (Leave-One-Out): CV 0.008194")
print(f"This (GroupKFold 5): CV {mse_overall:.6f}")

if mse_overall < 0.008194:
    print(f"\n✓ IMPROVEMENT: {(0.008194 - mse_overall) / 0.008194 * 100:.2f}% better than exp_035!")
else:
    print(f"\n✗ WORSE: {(mse_overall - 0.008194) / 0.008194 * 100:.2f}% worse than exp_035")

print(f"\n=== CV-LB RELATIONSHIP ANALYSIS ===")
print(f"If CV-LB relationship is LB = 4.23*CV + 0.0533:")
predicted_lb = 4.23 * mse_overall + 0.0533
print(f"Predicted LB: {predicted_lb:.4f}")
print(f"Best LB so far: 0.0877")
print(f"Target LB: 0.0347")